<a href="https://colab.research.google.com/github/wlrma0108/Stand_Alone_Deeplearning/blob/main/Stand_Alone%235.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install torch

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import argparse
import numpy as np
import time

In [4]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True, transform=transform)
trainset,valset = torch.utils.data.random_split(trainset, [40000,10000])
testset= torchvision.datasets.CIFAR10(root='./data',train=False, transform=transform)
partition = {'train':trainset, 'val':valset,'test': testset}

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [20]:
class MLP(nn.Module):
  def __init__(self,in_dim,out_dim,hid_dim,n_layer,act,dropout,use_bn,use_xavier):
    super(MLP,self).__init__()
    self.in_dim = in_dim
    self.out_dim=out_dim
    self.hid_dim=hid_dim
    self.n_layer=n_layer
    self.act=act
    self.dropout=dropout
    self.use_bn = use_bn
    self.use_xavier=use_xavier
  
    self.fc1 =nn.Linear(self.in_dim, self.hid_dim)
    self.linears=nn.ModuleList()
    self.bns=nn.ModuleList()
    for i in range(self.n_layer-1):
      self.linears.append(nn.Linear(self.hid_dim,self.hid_dim))
      if self.use_bn:
        self.bns.append(nn.BatchNorm1d(self.hid_dim))
    
    self.fc2 =nn.Linear(self.hid_dim, self.out_dim)

    if self.act =='relu':
      self.act = nn.ReLU()
    elif self.act =='tanh':
      self.act=nn.Tanh()
    elif self.act=='sigmoid':
      self.act=nn.Sigmoid()
    else:
      raise ValueError('no valid activation function selected!')

    self.dropout==nn.Dropout(self.dropout)
    if self.use_xavier:
      self.xavier_init()
  def forward(self, x):
    x= self.act(self.fc1(x))
    for i in range(len(self.linears)):
      x=self.act(self.linears[i](x))
      x=self.bns[i](x)
      x=self.dropout(x)
    x=self.fc2(x)
    return x
  def xavier_init(self):
    for linear in self.linears:
      nn.init.xavier_normal_(linear.weight)
      linear.bias.data.fill_(0.01)
net=MLP(3072,10,100,4,'relu',0.1,True,True)

In [6]:
def train(net, partition, optimizer, criterion, args):
  trainloader = torch.utils.data.DataLoader(partition['train'], batch_size=args.train_batch_size, shuffle=True, num_workers=2)
  net.train()

  correct=0
  total=0
  train_loss =0.0
  for i, data in enumerate(trainloader, 0):
    optimizer.zero_grad()
    inputs, labels =data
    inputs = inputs.view(-1,3072)
    outputs = net(inputs)

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss +=loss.item()
    _, predicted =torch.max(outputs.data,1)
    total+=labels.size(0)
    correct +=(predicted==labels).sum().item()
  train_loss =train_loss/len(trainloader)
  train_acc=100*correct/total
  return net, train_loss, train_acc



In [28]:
def validate(net, partition, criterion, args):
  valloader = torch.utils.data.DataLoader(partition['val'],batch_size=args.test_batch_size,shuffle=False,num_workers=2)
  net.eval()

  correct = 0
  total = 0
  val_loss = 0
  with torch.no_grad():
    for data in valloader:
      images, labels =data
      images = images.view(-1,3072)
      outputs = net(images)

      loss=criterion(outputs, labels)

      val_loss +=loss.item()
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted ==labels).sum().item()

    val_loss = val_loss/len(valloader)
    val_acc = 100*correct/total
  return val_loss, val_acc

In [8]:
def test(net, partition, args):
  testloader = torch.utils.data.DataLoader(partition['test'],batch_size=args.test_batch_size, shuffle=False,num_workers=2)

  net.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for data in testloader:
      images,labels = data
      images = images.view(-1, 3072)

      outputs = net(images)
      _, predicted = torch.max(outputs.data,1)
      total+= labels.size(0)
      correct +=(predicted ==labels).sum().item()
    test_acc=100 * correct/ total
  return test_acc

In [11]:
def experiment(partition, args):
  net=MLP(args.in_dim, args.out_dim, args.hid_dim, args.n_layer,  args.act,args.dropout,args.use_bn,args.use_xavier)

  criterion = nn.CrossEntropyLoss()
  if args.optim =='SGD':
    optimizer = optim.RMSprop(net.parameters(), lr=args.lr,weight_decay=args.l2)
  elif args.optim =='RMSprop':
    optimizer = optim.RMSprop(net.parameters(),)
  elif args.optim =='Adam':
    optimizer =optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
  else:
    raise ValueError('in-valid optimizer choice')
  
  for epoch in range(args.epoch):
    ts = time.time()
    net, train_loss, train_acc = train(net,partition, optimizer, criterion, args)
    val_loss, val_acc = validate(net, partition, criterion, args)
    te = time.time()
    print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
  test_acc = test(net, partition, args)
  return train_loss, val_loss, train_acc, val_acc,test_acc



In [ ]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.in_dim = 3072
args.out_dim = 10
args.hid_dim = 100
args.act = 'relu'

args.dropout = 0.2
args.use_bn = True
args.l2 = 0.00001
args.use_xavier = True

args.optim = 'RMSprop' 
args.lr = 0.0015
args.epoch = 10

args.train_batch_size = 256
args.test_batch_size = 1024

name_var1 = 'n_layer'
name_var2 = 'hid_dim'
list_var1 = [3, 3, 4]
list_var2 = [500, 300, 700]


for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
        result = experiment(partition, args)  

Namespace(act='relu', dropout=0.2, epoch=10, hid_dim=500, in_dim=3072, l2=1e-05, lr=0.0015, n_layer=3, optim='RMSprop', out_dim=10, test_batch_size=1024, train_batch_size=256, use_bn=True, use_xavier=True)
Epoch 0, Acc(train/val): 27.82/30.60, Loss(train/val) 2.79/2.80. Took 19.74 sec
Epoch 1, Acc(train/val): 37.91/33.96, Loss(train/val) 1.75/3.48. Took 19.64 sec
Epoch 2, Acc(train/val): 42.93/38.65, Loss(train/val) 1.60/35.72. Took 19.68 sec
Epoch 3, Acc(train/val): 45.55/39.73, Loss(train/val) 1.51/1.83. Took 19.80 sec
Epoch 4, Acc(train/val): 48.34/41.98, Loss(train/val) 1.44/27.54. Took 19.76 sec
Epoch 5, Acc(train/val): 50.92/43.46, Loss(train/val) 1.37/8.79. Took 19.57 sec
Epoch 6, Acc(train/val): 53.45/43.41, Loss(train/val) 1.30/17.35. Took 19.44 sec
Epoch 7, Acc(train/val): 55.63/47.89, Loss(train/val) 1.25/10.00. Took 19.67 sec
Epoch 8, Acc(train/val): 54.40/46.47, Loss(train/val) 1.28/2.06. Took 19.58 sec
Epoch 9, Acc(train/val): 56.64/46.08, Loss(train/val) 1.21/1.80. Took 

In [ ]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.in_dim = 3072
args.out_dim = 10
args.hid_dim = 100
args.act = 'relu'

args.dropout = 0.2
args.use_bn = True
args.l2 = 0.00001
args.use_xavier = True

args.optim = 'SGD' 
args.lr = 0.0015
args.epoch = 10

args.train_batch_size = 256
args.test_batch_size = 1024

name_var1 = 'n_layer'
name_var2 = 'hid_dim'
list_var1 = [3, 3, 4]
list_var2 = [500, 300, 700]


for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
        result = experiment(partition, args)  

In [ ]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.in_dim = 3072
args.out_dim = 10
args.hid_dim = 100
args.act = 'relu'

args.dropout = 0.2
args.use_bn = True
args.l2 = 0.00001
args.use_xavier = True

args.optim = 'ADAM' 
args.lr = 0.0015
args.epoch = 10

args.train_batch_size = 256
args.test_batch_size = 1024

name_var1 = 'n_layer'
name_var2 = 'hid_dim'
list_var1 = [3, 3, 4]
list_var2 = [500, 300, 700]


for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
        result = experiment(partition, args)  